In [1]:
# !pip install fastapi
# !pip install uvicorn
# !pip install pickle5
# !pip install pydantic
# !pip install scikit-learn
# !pip install requests
# !pip install pypi-json
# !pip install pyngrok
# !pip install nest-asyncio

  Using cached pickle5-0.0.11.tar.gz (132 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [17 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build

  Running setup.py clean for pickle5
Failed to build pickle5
  Running setup.py install for pickle5: started
  Running setup.py install for pickle5: finished with status 'error'



  creating build\lib.win-amd64-3.9
  creating build\lib.win-amd64-3.9\pickle5
  copying pickle5\pickle.py -> build\lib.win-amd64-3.9\pickle5
  copying pickle5\pickletools.py -> build\lib.win-amd64-3.9\pickle5
  copying pickle5\__init__.py -> build\lib.win-amd64-3.9\pickle5
  creating build\lib.win-amd64-3.9\pickle5\test
  copying pickle5\test\pickletester.py -> build\lib.win-amd64-3.9\pickle5\test
  copying pickle5\test\test_pickle.py -> build\lib.win-amd64-3.9\pickle5\test
  copying pickle5\test\test_picklebuffer.py -> build\lib.win-amd64-3.9\pickle5\test
  copying pickle5\test\__init__.py -> build\lib.win-amd64-3.9\pickle5\test
  running build_ext
  building 'pickle5._pickle' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for 

In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [3]:
app = FastAPI()

In [4]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [5]:
class model_input(BaseModel):
    
    Pregnancies : int
    Glucose : int
    BloodPressure : int
    SkinThickness : int
    Insulin : int
    BMI : float
    DiabetesPedigreeFunction : float
    Age : int

In [6]:
# loading the saved model
diabetes_model = pickle.load(open('trained_model_api.sav', 'rb'))

In [7]:
@app.post('/diabetes_prediction')
def diabetes_pred(input_parameters:model_input):
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)

    preg = input_dictionary['Pregnancies']
    glu = input_dictionary['Glucose']
    bp = input_dictionary['BloodPressure']
    skin = input_dictionary['SkinThickness']
    insulin = input_dictionary['Insulin']
    bmi = input_dictionary['BMI']
    dpf = input_dictionary['DiabetesPedigreeFunction']
    age = input_dictionary['Age']

    input_list = [preg,glu,bp,skin,insulin,bmi,dpf,age]

    prediction = diabetes_model.predict([input_list])

    if prediction[0] == 0:
        return 'THE PERSON IS NOT DIABETIC !!'
    else:
        return 'THE PERSON IS DIABETIC !!'

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [10144]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://d11e-2401-4900-313e-973f-adef-5394-71ca-ba6b.ngrok.io
INFO:     2401:4900:313e:973f:adef:5394:71ca:ba6b:0 - "POST /diabetes_prediction HTTP/1.1" 200 OK
INFO:     2401:4900:313e:973f:adef:5394:71ca:ba6b:0 - "POST /diabetes_prediction HTTP/1.1" 200 OK
